In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from rdkit import Chem
import numpy as np
import random
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from pysmiles import read_smiles
import pandas as pd
import logging
from tqdm import tqdm
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import torch
from torch.nn import Sequential as Seq, Linear, ReLU, CrossEntropyLoss
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing, GCNConv
from torch_geometric.utils import remove_self_loops, add_self_loops, degree
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
logging.getLogger('pysmiles').setLevel(logging.CRITICAL)  # Anything higher than warning

In [ ]:
def set_seed(seed =2019):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(2019)

In [ ]:
k=1
traindata = pd.read_csv(f'data/train_{k}_group_co2.csv')
valdata = pd.read_csv(f'data/val_{k}_group_co2.csv')
testdata = pd.read_csv('data/test_group_co2.csv')
fdata = pd.concat([traindata, valdata, testdata], ignore_index=True)
fdata.head()

In [ ]:
def uni_s(smile):
    mol = Chem.MolFromSmiles(smile)
    new_s = Chem.MolToSmiles(mol)
    return new_s

In [ ]:
elements=[]
for i in tqdm(range(len(fdata))):
    mol = Chem.MolFromSmiles(fdata['group'][i])
    mol = Chem.AddHs(mol)
    for i in range(len(mol.GetAtoms())):
        elements.append(mol.GetAtomWithIdx(i).GetSymbol())
elements = list(set(elements))
elements

In [ ]:
len(traindata), len(testdata), len(valdata)

In [ ]:
def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return [x == s for s in allowable_set]
def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(
            x, allowable_set))
    return [x == s for s in allowable_set]

In [ ]:
def atom_features(atom,
                  bool_id_feat=False,
                  explicit_H=False,
                  use_chirality=True):
    if bool_id_feat:
        return np.array([atom_to_id(atom)])
    else:
        results = one_of_k_encoding_unk(atom.GetSymbol(), ['H', 'F', 'C', 'B', 'S', 'Cl', 'N', 'Fe', 'P', 'Br', 'O']) + \
                  one_of_k_encoding_unk(atom.GetDegree(), [0,1,2,3,4,5])+[ atom.GetFormalCharge(), atom.GetNumRadicalElectrons(), atom.GetTotalNumHs()] + \
                  one_of_k_encoding_unk(atom.GetHybridization(), [
                    Chem.rdchem.HybridizationType.SP, Chem.rdchem.HybridizationType.SP2,
                    Chem.rdchem.HybridizationType.SP3, Chem.rdchem.HybridizationType.
                                        SP3D, Chem.rdchem.HybridizationType.SP3D2,'other'
                  ]) + [atom.GetIsAromatic()] #+ one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1,2,3,4])
        # In case of explicit hydrogen(QM8, QM9), avoid calling `GetTotalNumHs`
        if not explicit_H:
            results = results + one_of_k_encoding_unk(atom.GetTotalNumHs(),
                                                      [0, 1, 2, 3, 4])
        if use_chirality:
            try:
                results = results + one_of_k_encoding_unk(
                    atom.GetProp('_CIPCode'),
                    ['R', 'S']) + [atom.HasProp('_ChiralityPossible')]
            except:
                results = results + [False, False
                                     ] + [atom.HasProp('_ChiralityPossible')]

        return np.array(results)


In [ ]:
mol = Chem.MolFromSmiles(traindata['group'][0])
atom = mol.GetAtomWithIdx(4)
atom_features(atom, use_chirality=True)

In [ ]:
len(atom_features(atom))

In [ ]:
def element_to_onehot(s, x, y):
    mol = Chem.MolFromSmiles(s)
    out = []
    for i in range(0, len(mol.GetAtoms())):
        v = atom_features(mol.GetAtomWithIdx(i),use_chirality=True)
        v= list(v)
        v.append(x)
        v.append(y)
        v= np.array(v)
        out.append(v)
    return np.asarray(out)

In [ ]:
y_train = np.asarray(traindata['CO2-exp'])
y_val = np.asarray(valdata['CO2-exp'])
y_test = np.asarray(testdata['CO2-exp'])

In [ ]:
#process SMILES strings into graphs
def get_node_edge(tdata):
    nodes = []
    edge_index = []
    for i in range(len(tdata)):
        G_C = read_smiles(tdata['group'][i], explicit_hydrogen=False)
        feature = element_to_onehot(tdata['group'][i], tdata['T'][i]/fdata['T'].max(), tdata['P'][i]/fdata['P'].max())
        edges = np.asarray(G_C.edges)
        index = np.asarray([edges[:,0], edges[:,1]]) #reshape indices into shape [2, -1]
        nodes.append(feature)
        edge_index.append(index)
    return nodes, edge_index

In [ ]:
def get_data_loader(tdata, Y, batchsize, shuffle=True):
    node, edge = get_node_edge(tdata)
    mdata = list()
    for i in range(0, len(node)):
        x = torch.tensor(node[i], dtype=torch.float) #convert node features into torch tensor
        edges = torch.tensor(edge[i], dtype=torch.long) #convert edge index into torch tensor
        y = torch.tensor(Y[i], dtype=torch.float) #change shape of label and convert to tensor
        mdata.append(Data(x=x,edge_index=edges, y=y)) #add the Data object to the list of data
    loader = DataLoader(mdata, batch_size=batchsize, shuffle=shuffle)
    return loader

In [ ]:
train_loader = get_data_loader(traindata,y_train, 64, shuffle=False)
val_loader = get_data_loader(valdata, y_val, 64, shuffle=False)
test_loader = get_data_loader(testdata, y_test,64, shuffle=False)

In [ ]:
import dill
for i in range(1, 6):
    traindata = pd.read_csv(f'data/train_{i}_group_co2.csv')
    valdata = pd.read_csv(f'data/val_{i}_group_co2.csv')
    y_train = np.asarray(traindata['CO2-exp'])
    y_val = np.asarray(valdata['CO2-exp'])
    train_loader = get_data_loader(traindata,y_train,64, shuffle=False)
    val_loader = get_data_loader(valdata, y_val, 64, shuffle=False)
    with open( f'data/train_loader_{i}_co2.dump', 'wb') as f:
        dill.dump(train_loader, f)
    with open(f'data/val_loader_{i}_co2.dump', 'wb') as f:
        dill.dump(val_loader, f)

In [ ]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

In [ ]:
#define the message passing network
from torch_geometric.nn import global_add_pool, GraphConv
from torch_geometric.nn import global_mean_pool, BatchNorm
import torch.nn as nn
class Net(torch.nn.Module):
    def __init__(self, hidden_layer, hidden_size, drop_rate):
        super(Net, self).__init__()
        
        torch.manual_seed(12345)
        
        self.hidden_layer = hidden_layer
        self.drop_rate = drop_rate
        self.conv1 = GraphConv(37, hidden_size)
        if hidden_layer ==1:
            last_size = int(hidden_size/2)
            self.conv2 = GraphConv(hidden_size, int(hidden_size/2))
        else:
            last_size = int(hidden_size/4)
            self.conv2 = GraphConv(hidden_size, int(hidden_size/2))
            self.conv3 = GraphConv(int(hidden_size/2), int(hidden_size/4))
        self.lin1 = Linear(last_size, int(last_size/2))
        self.lin2 = Linear(int(last_size/2), 1)

    def forward(self, x, edge_index, batch, edge_weight=None):
        x = self.conv1(x, edge_index,edge_weight)
        x = F.relu(x)
        x = F.dropout(x, p=self.drop_rate, training=self.training)
        if self.hidden_layer ==1:
            x = self.conv2(x, edge_index,edge_weight)
            x = F.relu(x)
            x = F.dropout(x, p=self.drop_rate, training=self.training)
        else:
            x = self.conv2(x, edge_index, edge_weight)
            x = F.relu(x)
            x = F.dropout(x, p=self.drop_rate, training=self.training)
            x = self.conv3(x, edge_index,edge_weight)
            x = F.relu(x)
            x = F.dropout(x, p=self.drop_rate, training=self.training)
        x = global_add_pool(x, batch)
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        return x

In [ ]:
#set up device and create model
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #use CUDA if available
#model = Net().to(device) 
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4) 
loss_fn=F.mse_loss

In [ ]:
def train(model, loader, optimizer):
    model.train()
    for data in loader:
        data = data.to(device)  
        out = model(data.x, data.edge_index, data.batch)  
        loss = loss_fn(out, data.y.reshape(-1,1)) 
        loss.backward()  
        optimizer.step()  
        optimizer.zero_grad()

def test(model, loader):
    model.eval()
    predictions = []
    actul = []
    actul = np.array(actul)
    predictions= np.array(predictions)
    for data in loader:
        data = data.to(device) 
        out = model(data.x, data.edge_index, data.batch)  
        predictions= np.concatenate((predictions, out.cpu().detach().numpy().reshape(-1)))
        actul= np.concatenate((actul, data.y.cpu().detach().numpy()))
    return np.sqrt(mean_squared_error(actul, predictions)),r2_score(actul, predictions),

In [ ]:
class early_stopping:
    def __init__(self, patience):
        self.patience = patience
        self.counter = 0
        self.early_stop = False
        self.best_score = None
        self.loss_min = np.Inf
    def __call__(self, val_loss, model, file_name):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(model, file_name)
        elif score < self.best_score:
            self.counter += 1
            if self.counter > self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
            self.loss_min = val_loss
            self.model = model
            self.save_checkpoint(model, file_name)
    def save_checkpoint(self, model, file_name):
        torch.save(model.state_dict(), file_name)

In [ ]:
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK
from hyperopt.pyll.stochastic import sample
import numpy as np
import dill
space = {'hidden_size': hp.quniform('hidden_size',16, 1024, 16),
        'hidden_layers': hp.choice('hidden_layers',[1, 2]),
        'lr':hp.loguniform('r1_rate', np.log(0.0005), np.log(0.01)),
        'drop_rate': hp.quniform('drop_rate', 0, 0.5, 0.1),
        'weight_decay': hp.choice('weight_decay',[1e-06,1e-05, 1e-04, 1e-03,1e-02])}

In [ ]:
def fit(params):
    loss = []
    for i in range(1, 6):
        model = Net(params['hidden_layers'],params['hidden_size'],params['drop_rate']).to(device) 
        optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
        with open(f'data/train_loader_{i}_co2.dump', 'rb') as file:
            train_loader = dill.load(file)
        with open(f'data/val_loader_{i}_co2.dump', 'rb') as file:
            val_loader = dill.load(file)
        es=early_stopping(10)
        for epoch in range(6000):
            train(model,train_loader, optimizer)
            val_rmse, val_r2 = test(model, val_loader)
            es(val_rmse, model, 'data/GCN-co2.model')
            if es.early_stop:
                break
        loss.append(es.loss_min)
    return np.mean(loss)

In [ ]:
import pickle
def objective(params):
    global ITERATION
    ITERATION += 1
    for parameter_name in ['hidden_size', 'hidden_layers']:
        params[parameter_name] = int(params[parameter_name])
    val_loss= fit(params)
    loss = val_loss
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION])
    pickle.dump(bayes_trial, open("GCN_co2.p", "wb"))
    return {'loss': loss, 'params':params, 'iteration': ITERATION, 'status': STATUS_OK}

In [ ]:
import csv
out_file = 'GCN_co2_hyper.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)
writer.writerow(['loss','params','iteration'])
of_connection.close()

In [ ]:
tpe_algo = tpe.suggest
bayes_trial = Trials()

In [ ]:
#%%capture
from hyperopt.early_stop import no_progress_loss
global ITERATION
ITERATION =0
best = fmin(fn = objective, space =space, algo = tpe_algo, trials = bayes_trial,
            early_stop_fn=no_progress_loss(100),max_evals=3000, rstate= np.random.RandomState(50)) 

In [ ]:
result = pd.read_csv('GCN_co2_hyper.csv')
result.sort_values('loss', ascending= True, inplace = True)
result.reset_index(drop = True, inplace =True)
result.head()

In [ ]:
import ast
params = ast.literal_eval(result['params'][0])                                                                                                                                                                                                                                                               
params

In [ ]:
def test_hh(loader):
    model.eval()
    predictions = []
    actul = []
    actul = np.array(actul)
    predictions= np.array(predictions)
    for data in loader:
        data = data.to(device) 
        out = model(data.x, data.edge_index, data.batch)  
        predictions= np.concatenate((predictions, out.cpu().detach().numpy().reshape(-1)))
        actul= np.concatenate((actul, data.y.cpu().detach().numpy()))
    return actul, predictions

In [ ]:
import dill
for i in range(1, 6):
    model = Net(params['hidden_layers'],params['hidden_size'],params['drop_rate']).to(device) 
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'], weight_decay=params['weight_decay'])
    with open(f'data/train_loader_{i}_co2.dump', 'rb') as file:
        train_loader = dill.load(file)
    with open(f'data/val_loader_{i}_co2.dump', 'rb') as file:
        val_loader = dill.load(file)
    es=early_stopping(100)
    for epoch in range(6000):
        train(model,train_loader, optimizer)
        val_rmse, val_r2 = test(model, val_loader)
        es(val_rmse, model, f'data/GCN-co2_{i}.model')
        if es.early_stop:
            break

In [ ]:
len(testdata)

In [ ]:
true_pred = np.zeros(shape=(1324,))
for i in range(1, 6):
    model = Net(params['hidden_layers'],params['hidden_size'],params['drop_rate']).to(device) 
    model.load_state_dict(torch.load(f'data/GCN-co2_{i}.model'))
    y_test, y_test_pred = test_hh(test_loader)
    true_pred = true_pred + y_test_pred
true_pred=true_pred/5